In [20]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
import requests
from scipy import stats
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import RFE
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
# from tensorflow.keras.layers import *
# import tensorflow as tf

In [21]:
train_data = pd.read_csv('./X_train.csv')
train_targets = pd.read_csv('./y_train.csv')
test_data = pd.read_csv('./X_test.csv')

In [22]:
train_data = train_data.drop(['id', 'Moons'], axis=1)
test_data = test_data.drop(['id', 'Moons'], axis=1)

In [23]:
train_data.head()

,wrythm_Feature_1,wrythm_Feature_2,wrythm_Feature_3,wrythm_Feature_4,wrythm_Feature_5,wrythm_Feature_6,wrythm_Feature_7,wrythm_Feature_8,wrythm_Feature_9,wrythm_Feature_10,...,3b1-signal_Feature_42,3b1-signal_Feature_43,3b1-signal_Feature_44,3b1-signal_Feature_45,3b1-signal_Feature_46,3b1-signal_Feature_47,3b1-signal_Feature_48,3b1-signal_Feature_49,3b1-signal_Feature_50,3b1-signal_Feature_51
0,0.67,0.67,0.17,0.83,0.33,0.83,0.83,0.50,0.50,1.00,...,0.33,0.67,0.50,1.00,0.33,0.17,0.67,0.33,0.17,0.50
1,0.67,0.50,0.33,0.17,0.33,0.33,0.67,0.50,0.83,0.50,...,0.33,0.67,0.50,0.50,0.67,0.67,0.83,0.50,0.00,0.33
2,1.00,0.50,0.50,0.50,0.33,0.67,0.50,0.50,0.50,0.67,...,0.67,0.67,1.00,0.50,0.83,0.67,0.33,0.67,0.50,0.17
3,0.33,0.33,0.33,1.00,0.33,0.67,0.67,0.33,0.33,0.50,...,0.83,0.50,0.33,0.50,0.83,0.33,0.50,0.83,0.67,0.67
4,0.17,0.33,0.00,0.17,0.50,0.33,0.83,0.67,0.00,0.33,...,0.00,0.50,0.67,0.33,0.33,0.50,0.50,0.17,0.67,0.67


In [24]:
train_targets.head()

,id,Moons,target_w,target_r,target_g,target_b
0,37745,0,0.50,0.33,0.33,0.17
1,69201,0,0.50,0.50,0.50,0.67
2,96831,0,0.33,0.50,0.50,0.50
3,185617,0,0.67,0.67,0.50,0.67
4,38148,0,0.50,0.33,0.50,0.33


In [25]:
test_data.head()

,wrythm_Feature_1,wrythm_Feature_2,wrythm_Feature_3,wrythm_Feature_4,wrythm_Feature_5,wrythm_Feature_6,wrythm_Feature_7,wrythm_Feature_8,wrythm_Feature_9,wrythm_Feature_10,...,3b1-signal_Feature_42,3b1-signal_Feature_43,3b1-signal_Feature_44,3b1-signal_Feature_45,3b1-signal_Feature_46,3b1-signal_Feature_47,3b1-signal_Feature_48,3b1-signal_Feature_49,3b1-signal_Feature_50,3b1-signal_Feature_51
0,0.17,0.50,0.50,0.50,0.17,0.83,0.50,1.00,0.50,0.67,...,0.67,0.50,0.50,0.33,0.33,0.67,0.50,0.33,0.50,0.67
1,0.50,0.67,0.33,0.33,0.33,0.67,0.33,0.67,0.50,0.50,...,0.50,0.33,0.50,0.50,0.50,0.67,0.33,0.33,0.50,0.67
2,0.33,0.83,0.17,0.00,0.67,0.00,0.67,0.67,0.67,0.17,...,0.17,0.33,0.50,0.67,0.83,0.33,0.67,0.33,0.67,0.50
3,0.33,0.33,0.83,0.67,0.33,0.67,0.17,0.67,0.67,0.33,...,0.50,0.17,0.33,0.50,0.50,0.50,0.50,0.50,0.83,0.83
4,0.83,0.67,0.17,0.67,0.17,0.83,0.83,0.50,0.67,0.67,...,0.83,0.83,0.50,0.83,0.67,0.83,0.33,0.50,0.50,0.33


# Modeling



In [26]:
# model = xgb.XGBRegressor(objective='reg:squarederror', max_depth=5, learning_rate=0.01, n_estimators=2000, n_jobs=-1, colsample_bytree=0.5)

In [27]:
def model(data, target):
    X, y = data, target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LGBMRegressor(num_leaves=300)

    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    scorer(y_test, pred)

    return model

In [28]:
def scorer(y_test, y_pred):
    score = (stats.spearmanr(y_test, y_pred)*100)[0]
    print('Score as calculated for the leader board (っಠ‿ಠ)っ {}'.format(score))
    # print('accuracy_score: {}'.format(accuracy_score(y_test, y_pred)))

In [29]:
model_target_r= model(train_data, train_targets.target_r)
model_target_g= model(train_data, train_targets.target_g)
model_target_b= model(train_data, train_targets.target_b)
model_target_w = model(train_data, train_targets.target_w)

Score as calculated for the leader board (っಠ‿ಠ)っ 0.4655003961024462
Score as calculated for the leader board (っಠ‿ಠ)っ 0.6879593604588499
Score as calculated for the leader board (っಠ‿ಠ)っ 0.7580540255199344
Score as calculated for the leader board (っಠ‿ಠ)っ 0.036476921564902155


In [30]:
#because target w is not really good so lets see with another model for target 2
X_train, X_test, y_train, y_test = train_test_split(train_data, train_targets.target_w, test_size=0.2, random_state=42)

model_2 =xgb.XGBRegressor(objective='reg:squarederror', max_depth=5, learning_rate=0.01, n_estimators=2000, n_jobs=-1, colsample_bytree=0.5)
model_2.fit(X_train, y_train)
pred = model_2.predict(X_test)
scorer(y_test, pred)

Score as calculated for the leader board (っಠ‿ಠ)っ 0.04771133101028098


In [47]:
def linear_regression(train_data, train_targets):
    X_train, X_test, y_train, y_test = train_test_split(train_data, train_targets, test_size=0.2, random_state=42)

    model =LinearRegression()
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    scorer(y_test, pred)

In [48]:
model_lr_r= linear_regression(train_data, train_targets.target_r)
model_lr_g= linear_regression(train_data, train_targets.target_g)
model_lr_b= linear_regression(train_data, train_targets.target_b)
model_lr_w = linear_regression(train_data, train_targets.target_w)

Score as calculated for the leader board (っಠ‿ಠ)っ 0.10430481521941734
Score as calculated for the leader board (っಠ‿ಠ)っ 0.13753997615051125
Score as calculated for the leader board (っಠ‿ಠ)っ 0.15420047847274873
Score as calculated for the leader board (っಠ‿ಠ)っ 0.04278190833693602


In [ ]:
model_3 = model(train_data, train_targets.target_w)

# Features Selection

## Using RFE

In [32]:
# def rfe_selection(test_data_input, train_data_input, target):

#     print('\nSelecting features...')
#     recur = RFE(estimator = model, n_features_to_select = 15)
#     features = recur.fit(train_data_input, target)

#     # print(f'Number of features: {features.n_features_} ')
#     # print(f'Selected Features are: {features.support_}')
#     # print(f'Features rangking is as follows: {features.ranking_}')
#     new_features = []

#     print('Create new features...')
#     for i, v in enumerate(features.support_):
#         if v == True:
#             new_features.append(train_data_input.columns[i])
    
#     new_train_data = train_data_input[new_features]
#     #print(new_train_data.columns)
#     new_test_data = test_data_input[new_features]
#     #print(new_test_data.columns)

#     # print('Modeling with new features...')
#     # model = xg_boost_hackathon(model, new_train_data, target)


#     return new_test_data, new_train_data



In [33]:
# test_data_r, train_data_r = rfe_selection(test_data, train_data, train_targets.target_r)
# test_data_g, train_data_g= rfe_selection(test_data, train_data, train_targets.target_g)
# test_data_b, train_data_b= rfe_selection(test_data, train_data,train_targets.target_b)

In [34]:
# test_data_r.columns

In [35]:
# train_data_r.columns

In [36]:
# rfe_model_r= xg_boost_hackathon(train_data_r, train_targets.target_r)
# rfe_model_g= xg_boost_hackathon(train_data_g, train_targets.target_g)
# rfe_model_b= xg_boost_hackathon(train_data_b, train_targets.target_b)

## Using Chi Score

In [37]:
# def feature_selection(model, train_data):
#     feature_selection = []
#     i = 0
#     for feature_importance in model.feature_importances_:
#         if feature_importance >= 0.005:
#             #print(feature_importance)
#             feature_selection.append(train_data.columns[i])
#         i+=1

#     train_data_selection = train_data[feature_selection]
    
#     return train_data_selection

In [38]:
# feature_model_r = feature_selection(model_target_r, train_data)
# # because the feature selection for 3 models is same therfore I comment the rest model
# feature_model_g = feature_selection(model_target_g, train_data)
# feature_model_b = feature_selection(model_target_b,train_data)

# Submission files

In [39]:
test_data = pd.read_csv('./X_test.csv')
submission = test_data[['id', 'Moons']]

In [40]:
submission['target_w'] = np.nan
submission['target_r'] = np.nan
submission['target_g'] = np.nan
submission['target_b'] = np.nan

C:\Users\ASUS\AppData\Local\Temp\ipykernel_14456\1682246736.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['target_w'] = np.nan
C:\Users\ASUS\AppData\Local\Temp\ipykernel_14456\1682246736.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['target_r'] = np.nan
C:\Users\ASUS\AppData\Local\Temp\ipykernel_14456\1682246736.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [41]:
submission

,id,Moons,target_w,target_r,target_g,target_b
0,31324,361,NaN,NaN,NaN,NaN
1,253769,361,NaN,NaN,NaN,NaN
2,312932,361,NaN,NaN,NaN,NaN
3,319168,361,NaN,NaN,NaN,NaN
4,319189,361,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
10660,148996,373,NaN,NaN,NaN,NaN
10661,195471,373,NaN,NaN,NaN,NaN
10662,75924,373,NaN,NaN,NaN,NaN
10663,283674,373,NaN,NaN,NaN,NaN


In [42]:
test_data = test_data.drop(['id', 'Moons'], axis=1)

submission['target_w'] = model_3.predict(test_data)
submission['target_r'] = model_target_r.predict(test_data)
submission['target_g'] = model_target_g.predict(test_data)
submission['target_b'] = model_target_b.predict(test_data)

In [43]:
submission

,id,Moons,target_w,target_r,target_g,target_b
0,31324,361,0.500911,0.532434,0.495687,0.494677
1,253769,361,0.497578,0.495948,0.452800,0.451568
2,312932,361,0.496775,0.595202,0.575724,0.624430
3,319168,361,0.500803,0.472909,0.467508,0.452056
4,319189,361,0.502995,0.526591,0.461154,0.366384
...,...,...,...,...,...,...
10660,148996,373,0.494587,0.524371,0.518799,0.422672
10661,195471,373,0.506421,0.536208,0.526690,0.449922
10662,75924,373,0.499675,0.512222,0.504378,0.541412
10663,283674,373,0.494909,0.461494,0.398339,0.477803


In [44]:
API_KEY = "" # <- enter your API key HERE

## new version
r = requests.post(
    "https://api.tournament.crunchdao.com/v2/submissions",
    params={
        "apiKey": API_KEY,
    },
    files={
        "file": ("x", submission.to_csv(index=False).encode('ascii'))
    },
)

if r.status_code == 200:
    print("Submission submitted.")
elif r.status_code == 423:
    print("ERR: Submissions are close")
    print("You can only submit during rounds eg: Friday 7pm GMT+1 to Sunday midnight GMT+1.")
    print("Or the server is currently crunching the submitted files, please wait some time before retrying.")
elif r.status_code == 422:
    print("ERR: API Key is missing or empty")
    print("Did you forget to fill the API_KEY variable?")
elif r.status_code == 400:
    print("ERR: The file must not be empty")
    print("You have send a empty file.")
elif r.status_code == 401:
    print("ERR: Your email hasn't been verified")
    print("Please verify your email or contact a cruncher.")
elif r.status_code == 409:
    print("ERR: Duplicate submission")
    print("Your work has already been submitted with the same exact results, if you think that this a false positive, contact a cruncher.")
    print("MD5 collision probability: 1/2^128 (source: https://stackoverflow.com/a/288519/7292958)")
elif r.status_code == 429:
    print("ERR: Too many submissions")
else:
    print("ERR: Server returned: " + str(r.status_code))
    print("Ouch! It seems that we were not expecting this kind of result from the server, if the probleme persist, contact a cruncher.")

Submission submitted.
